### Ideen Use Case
- Gesetzbuch ZGB
- Kochbuch
- LLM Agent generiert Werbetexe, Social Media Posts

### FWO-Assistent

- Hilft bei der Erstellung von Texten für LinkedIn, Instagram, WhatsApp etc.

- Auf Basis von vergangenen Posts

In [1]:
import os
import langchain

LLM_MODEL = "openai/gpt-oss-20b:free"
LLM_MODEL = "gpt-oss-120b"
LLM_TEMPERATURE = 0.4
BASE_URL = "https://openrouter.ai/api/v1"
LLM_API_KEY = os.environ["OPEN_ROUTER_API_KEY"]
USER_PROMPT="Generiere mir einen Instagrampost"

ModuleNotFoundError: No module named 'langchain'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    base_url=BASE_URL,
    api_key=LLM_API_KEY,
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

FWO_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "Du bist der FWO-Assistent (Fachschaft Wirtschaft Olten, FHNW). "
     "Bevor du schreibst, beziehst du dich IMMER auf: "
     "1) Kampagnenkonzept (Ziel, Kernbotschaft, Mehrwert) "
     "2) Redaktionsplan (Phase, wiederkehrende Formate, Timing). "
     "Vermeide Genderung und gebrauche bspw. Lehrperson anstatt Lehrer*innen "
     "Generiere jeweils immer zwei Vorschläge zu jeder Plattform (Instagram, WhatsApp, LinkedIn)"
     "Vermeide Floskeln, schreibe aktiv und konkret."
     "If you are unsure or the answer isn't in the context, say that you don't know.\n\n"
    ),
    ("user",
     "User: {text}")
])


In [ ]:
chain = FWO_PROMPT | llm

In [ ]:
from langchain_core.output_parsers import StrOutputParser

USER_PROMPT="Generiere mir einen Post für die PubTour am 16. Oktober"

chain_simple = (FWO_PROMPT | llm | StrOutputParser())
result = chain_simple.invoke(USER_PROMPT)
print(result)

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

# List of PDF file paths
pdf_files = [
    "SoMeKonzept (1).pdf"
]

# Load all documents
all_pages_pdf = []
for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    pages = loader.load()
    all_pages_pdf.extend(pages)

print(f"Loaded {len(all_pages_pdf)} pages from {len(pdf_files)} pdf documents.")


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# get both websites and pdfs together
all_docs = all_pages_pdf

# define the splitter and strategy
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = splitter.split_documents(all_docs)

In [ ]:
import numpy as np

lengths = [len(s.page_content) for s in splits]
print(f"Initial documents: {len(all_docs)}")
print(f"Total chunks: {len(splits)}")
print(f"Avg length: {np.mean(lengths):.1f}")
print(f"Min: {np.min(lengths)}, Max: {np.max(lengths)}")

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    normalize_L2=True
)

vector_store.add_documents(documents=splits)

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
docs = retriever.get_relevant_documents("Hashtags für Instagram")
for i, d in enumerate(docs, 1):
    print(f"--- Retrieved doc {i} ---")
    print(d.page_content[:400], "...\n")

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | response_llm
    | output_parser
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

USER_PROMPT="Generiere mir einen Post für die PubTour am 16. Oktober"

result = chain.invoke(USER_PROMPT)
print(result)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56a0a349-7f2e-43e5-8d52-5ded467f6e9c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>